In [0]:
%pip install bokeh

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from pyspark.sql.functions import *

In [0]:
#Data reading and cleaning
file_year = '2021'
clinicaltrial_df = spark.read.options(delimiter="|", header = "True"). csv("/FileStore/tables/clinicaltrial_" +file_year+ ".csv")
mesh_df = spark.read.options(header = "True").csv("dbfs:/FileStore/tables/mesh.csv")
pharma_df = spark.read.options(delimiter=",", header = "True", inferschema = 'True'). csv("/FileStore/tables/pharma.csv")

In [0]:
#1. The number of studies in the dataset. You must ensure that you explicitly check distinct studies.
clinicaltrial_df.distinct().count()

Out[3]: 387261

In [0]:
#2. You should list all the types (as contained in the Type column) of studies in the dataset along with the frequencies of each type. 
# These should be ordered from most frequent to least frequent.

type_df = clinicaltrial_df.groupBy(clinicaltrial_df.Type).count().sort(desc('count'))
type_df.display()

Type,count
Interventional,301472
Observational,77540
Observational [Patient Registry],8180
Expanded Access,69


In [0]:
#3. The top 5 conditions (from Conditions) with their frequencies.

conditions_df = clinicaltrial_df.filter(clinicaltrial_df.Conditions != 'null')\
                                .select(explode(split(clinicaltrial_df.Conditions, ',')).alias('Conditions_Cleaned'))\
                                .groupBy('Conditions_Cleaned')\
                                .count()\
                                .sort(desc('count'))
conditions_df.show(5)

+------------------+-----+
|Conditions_Cleaned|count|
+------------------+-----+
|         Carcinoma|13389|
| Diabetes Mellitus|11080|
|         Neoplasms| 9371|
|  Breast Neoplasms| 8640|
|          Syndrome| 8032|
+------------------+-----+
only showing top 5 rows



In [0]:
#4. Each condition can be mapped to one or more hierarchy codes. The client wishes to know the 
# 5 most frequent roots (i.e. the sequence of letters and numbers before the first full stop) after this is done.

conditions_df_without_count = clinicaltrial_df.filter(clinicaltrial_df.Conditions != 'null')\
                                              .select(explode(split(clinicaltrial_df.Conditions, ',')).alias('Conditions_Cleaned'))

condition_mesh_df = conditions_df_without_count.join(mesh_df,conditions_df_without_count.Conditions_Cleaned ==  mesh_df.term,"inner")

condition_code_df = condition_mesh_df.withColumn("Codes", condition_mesh_df.tree.substr(1,3))
code_count = condition_code_df.groupBy('Codes')\
                              .count()\
                              .sort(desc('count'))

code_count.show(10)

+-----+------+
|Codes| count|
+-----+------+
|  C04|143994|
|  C23|136079|
|  C01|106674|
|  C14| 94523|
|  C10| 92310|
|  C06| 85646|
|  C08| 70720|
|  C13| 42599|
|  C18| 41276|
|  C12| 40161|
+-----+------+
only showing top 10 rows



In [0]:
#5. Find the 10 most common sponsors that are not pharmaceutical companies, along with 
# the number of clinical trials they have sponsored. Hint: For a basic implementation, 
#you can assume that the Parent Company column contains all possible pharmaceutical companies.

parent_company_df = pharma_df.select('Parent_Company').distinct()
pharma_sponsors = clinicaltrial_df.select('Sponsor')\
                                  .groupBy('Sponsor')\
                                  .count()\
                                  .sort(desc('count'))
non_pharma_sponsors= pharma_sponsors.join(parent_company_df,pharma_sponsors.Sponsor == parent_company_df.Parent_Company,"left_anti")
non_pharma_sponsors.show(10, False)

+---------------------------------------+-----+
|Sponsor                                |count|
+---------------------------------------+-----+
|National Cancer Institute (NCI)        |3218 |
|M.D. Anderson Cancer Center            |2414 |
|Assistance Publique - Hôpitaux de Paris|2369 |
|Mayo Clinic                            |2300 |
|Merck Sharp & Dohme Corp.              |2243 |
|Assiut University                      |2154 |
|Novartis Pharmaceuticals               |2088 |
|Massachusetts General Hospital         |1971 |
|Cairo University                       |1928 |
|Hoffmann-La Roche                      |1828 |
+---------------------------------------+-----+
only showing top 10 rows



In [0]:
#6. Plot number of completed studies each month in a given year – for the submission dataset, 
# the year is 2021. You need to include your visualization as well as a table of all the values 
#you have plotted for each month.

completion_status_df = clinicaltrial_df.withColumn('Completion_date', to_date(clinicaltrial_df['Completion'], 'MMM yyyy'))
completion_filter = completion_status_df.filter((year('Completion_date') == file_year) & (col('Status') == 'Completed'))
completion_months_count = completion_filter.select(date_format('Completion_date', 'MMM').alias('Completion_month'))\
                                           .groupBy('Completion_month')\
                                           .count()\
                                           .sort(to_date('Completion_month', 'MMM'))
completion_months_count.display()

Completion_month,count
Jan,1131
Feb,934
Mar,1227
Apr,967
May,984
Jun,1094
Jul,819
Aug,700
Sep,528
Oct,187


In [0]:
import pandas as pd
from bokeh.resources import INLINE,CDN
import bokeh.io
from bokeh.embed import file_html, components
from bokeh.models import ColumnDataSource, FactorRange, HoverTool
from bokeh.plotting import figure, output_file

pandas_completion_df = completion_months_count.toPandas()

months = pandas_completion_df['Completion_month'].tolist()
counts = pandas_completion_df['count'].tolist()

data = {"x_axis": months,
        "y_axis": counts}

source = ColumnDataSource(data = data)

p = figure(x_range = months, plot_height =250, toolbar_location = None, title ="Number of trials Completed month-wise", tools="hover",
           tooltips=[("Months", "@x_axis"),("Count","@y_axis")])
p.vbar(x="x_axis",source = source, top="y_axis", width=0.5, color="#bc8f8f")
p.xgrid.grid_line_color = None
p.y_range.start = 0

html = file_html(p,CDN,'plot')
displayHTML(html)

<!DOCTYPE html>
 
 
 
 
 
 plot